# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
!pip install folium


# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ushuaia,-54.8000,-68.3000,288.96,59,40,2.57,AR,1707064326
1,1,guaranda,-1.6000,-79.0000,290.30,82,100,2.53,EC,1707064506
2,2,aylesbury,51.8167,-0.8146,286.25,88,100,8.53,GB,1707064506
3,3,minzhu,43.7286,127.3447,249.17,99,34,1.52,CN,1707064506
4,4,port-aux-francais,-49.3500,70.2167,277.90,73,71,11.54,TF,1707064507


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

import folium

# Create a base map centered at a specific location (e.g., the first city in your DataFrame)
base_map = folium.Map(location=[city_data_df['Lat'].iloc[0], city_data_df['Lng'].iloc[0]], zoom_start=4)

# Add a marker for each city with the size proportional to the humidity
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 5,  
        popup=f"{row['City']}\nHumidity: {row['Humidity']}%",
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7
    ).add_to(base_map)

# Save the map as an HTML file or display it
base_map.save("city_map.html") 
base_map  


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

# Filter cities with 0 cloudiness
cities_no_cloudiness = city_data_df[city_data_df['Cloudiness'] == 0]

# Display the filtered DataFrame
cities_no_cloudiness

# Drop any rows with null values
cities_cleaned = cities_no_cloudiness.dropna()

# Display the cleaned DataFrame
cities_cleaned.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,farah,32.5000,62.5000,278.60,26,0,0.55,AF,1707064507
32,32,fort bragg,35.1390,-79.0060,284.37,50,0,4.63,US,1707064513
34,34,badger,64.8000,-147.5333,242.79,67,0,0.00,US,1707064513
37,37,griffith,41.5284,-87.4237,280.01,78,0,2.06,US,1707064514
43,43,bethel,41.3712,-73.4140,278.45,55,0,4.12,US,1707064245


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display the DataFrame with the new column
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
0,ushuaia,AR,-54.8000,-68.3000,59,
1,guaranda,EC,-1.6000,-79.0000,82,
2,aylesbury,GB,51.8167,-0.8146,88,
3,minzhu,CN,43.7286,127.3447,99,
4,port-aux-francais,TF,-49.3500,70.2167,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
# Set parameters to search for a hotel
radius = 500
params = {
    'type': 'hotel',
    'radius': radius,
    'apiKey': geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["bias"] = f"circle:{radius}@{latitude},{longitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places/search"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Check if the API request was successful (status code 200)
    if response.status_code == 200:
        data = response.json()

        # Assume your function returns a list of hotel names
        name_address = [result.get('properties', {}).get('name', 'No hotel found') for result in data['features']]

        # Grab the first hotel from the results and store the name in the hotel_df DataFrame
        try:
            hotel_df.loc[index, "Hotel Name"] = name_address[0]
        except (IndexError, KeyError):
            # If no hotel is found, set the hotel name as "No hotel found".
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"


    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()


Starting hotel search
ushuaia - nearest hotel: 
guaranda - nearest hotel: 
aylesbury - nearest hotel: 
minzhu - nearest hotel: 
port-aux-francais - nearest hotel: 
mont-de-marsan - nearest hotel: 
emerald - nearest hotel: 
farah - nearest hotel: 
iqaluit - nearest hotel: 
west island - nearest hotel: 
adamstown - nearest hotel: 
waitangi - nearest hotel: 
grytviken - nearest hotel: 
halifax - nearest hotel: 
albany - nearest hotel: 
baijiantan - nearest hotel: 
yenisehir - nearest hotel: 
puerto natales - nearest hotel: 
varberg - nearest hotel: 
hermanus - nearest hotel: 
bilibino - nearest hotel: 
thompson - nearest hotel: 
olonkinbyen - nearest hotel: 
stanley - nearest hotel: 
robore - nearest hotel: 
constantia - nearest hotel: 
arvika - nearest hotel: 
fort st. john - nearest hotel: 
sake - nearest hotel: 
oriximina - nearest hotel: 
blackmans bay - nearest hotel: 
sao lourenco do sul - nearest hotel: 
fort bragg - nearest hotel: 
hotan - nearest hotel: 
badger - nearest hotel: 


In [ ]:
%%capture --no-display
import folium

# Create a map centered at the coordinates of the first city in the DataFrame
map_center = [hotel_df.iloc[0]['Lat'], hotel_df.iloc[0]['Lng']]
my_map = folium.Map(location=map_center, zoom_start=12)  

# Add markers for each hotel in the DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    hotel_name = row['Hotel Name']

    # Add a marker for each hotel
    folium.Marker(
        location=[latitude, longitude],
        popup=f"Hotel: {hotel_name}",
        icon=folium.Icon(color='blue')
    ).add_to(my_map)

# Save the map to an HTML file
my_map.save("hotel_map.html")
